In [ ]:
import torch
from torchvision.models import resnet50, ResNet50_Weights
from PIL import Image
import os

# 1. Load the Pre-trained Model
print("Loading ResNet50 model...")
weights = ResNet50_Weights.DEFAULT
model = resnet50(weights=weights)
model.eval()
preprocess = weights.transforms()

# 2. Get Model Categories
class_names = weights.meta["categories"]

# 3. Find Uploaded Images
# We scan the current directory for any .png file
current_directory = "."
files = [f for f in os.listdir(current_directory) if f.endswith(".png")]

if len(files) == 0:
    print("❌ No images found! Please drag and drop your .png files into the Files sidebar on the left.")
else:
    print(f"✅ Found {len(files)} images to test.\n")

    print(f"{'File Name':<20} | {'True Species':<20} | {'Model Prediction':<20} | {'Result'}")
    print("-" * 90)

    correct_count = 0

    for filename in files:
        # 4. Extract "True Label" from Filename
        # Assuming filename is "Lion_1.png", we split by "_" to get "Lion"
        try:
            true_label = filename.split("_")[0]
        except:
            true_label = "Unknown"

        # 5. Load and Preprocess
        img_path = os.path.join(current_directory, filename)
        try:
            img = Image.open(img_path).convert("RGB")
        except:
            print(f"Error reading {filename}")
            continue

        batch = preprocess(img).unsqueeze(0)

        # 6. Inference
        with torch.no_grad():
            prediction = model(batch).squeeze(0).softmax(0)

        class_id = prediction.argmax().item()
        predicted_label = class_names[class_id]

        # 7. Check Match (Fuzzy logic for uppercase/lowercase)
        # We check if the True Label is inside the Predicted Label (e.g., "Lion" in "lion")
        if true_label.lower() in predicted_label.lower() or predicted_label.lower() in true_label.lower():
            result = "✅ Match"
            correct_count += 1
        else:
            result = "❌ Miss"

        print(f"{filename:<20} | {true_label:<20} | {predicted_label:<20} | {result}")

    # 8. Final Score
    if len(files) > 0:
        accuracy = (correct_count / len(files)) * 100
        print("-" * 90)
        print(f"Final Accuracy: {accuracy:.2f}%")

Loading ResNet50 model...
✅ Found 14 images to test.

File Name            | True Species         | Model Prediction     | Result
------------------------------------------------------------------------------------------
AsianGoldenCat_1.png | AsianGoldenCat       | tabby                | ❌ Miss
CloudedLeopard_10.png | CloudedLeopard       | tiger                | ❌ Miss
Jaguarundi_2.png     | Jaguarundi           | lynx                 | ❌ Miss
Leopard_3.png        | Leopard              | leopard              | ✅ Match
Puma_4.png           | Puma                 | cougar               | ❌ Miss
Jaguar_3.png         | Jaguar               | jaguar               | ✅ Match
EurasianLynx_1.png   | EurasianLynx         | lynx                 | ✅ Match
IberianLynx_5.png    | IberianLynx          | lynx                 | ✅ Match
PampasCat_1.png      | PampasCat            | tiger cat            | ❌ Miss
AfricanGoldenCat_1.png | AfricanGoldenCat     | Egyptian cat         | ❌ Miss
Lion_5.png  